# Run GCN on input data

In [ ]:
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
import pandas as pd
import numpy as np
from scipy.io import mmread
import networkx as nx
import torch_geometric
import matplotlib
import matplotlib.pyplot as plt
import umap
from sklearn.preprocessing import StandardScaler

## Create initial graph

In [ ]:
df_bed = pd.read_csv('../output/EB1/test_gene_TF.bed', sep='\t', header=None)
df_bed.columns = ['chr_motif', 'start_motif', 'end_motif', 'motif_name', 'num_motif', 'strand_motif', 'gene_motif', 'chr_peak', 'start_peak', 'end_peak', 'gene_name', 'num_peak', 'strand_peak', 'chr_TSS', 'start_TSS', 'end_TSS', 'score']

In [ ]:
df_bed.iloc[0:10, ]

In [ ]:
motif_list = list(set(df_bed['motif_name']))
gene_list = list(set(df_bed['gene_name']))

In [ ]:
motif_list = [motif for motif in motif_list if motif.split('(')[0].split(':')[0].upper() in gene_list]
TF_list = [motif.split('(')[0].split(':')[0].upper() for motif in motif_list]
d_TF = dict(zip(motif_list, TF_list))

In [ ]:
# Only keeping TFs that are within the input gene list

df_bed = df_bed[df_bed['motif_name'].isin(motif_list)]
df_bed['TF_name'] = df_bed['motif_name'].map(d_TF)

In [ ]:
df_bed.iloc[0:10, ]

In [ ]:
mat = mmread('../data/EB1_count.mtx')
mat_np = np.array(mat.todense())

In [ ]:
df_obs = pd.read_csv('../data/EB1_obs.txt', header=0, sep='\t')
df_var = pd.read_csv('../data/EB1_var.txt', header=0, sep='\t')

In [ ]:
# Ensure that every gene in the graph can get the initial feature from scRNA-seq

df_bed = df_bed[(df_bed['TF_name'].isin(df_var['gene_short_name'])) & (df_bed['gene_name'].isin(df_var['gene_short_name']))]

In [ ]:
# Get links between TFs and genes, aggregate them and take the average

df_link = df_bed[['TF_name', 'gene_name', 'score']]
df_link = df_link[df_link['score'] != np.inf]
df_network = df_link.groupby(['TF_name', 'gene_name'], as_index=False).agg({'score': 'mean'})
TF_list = df_network['TF_name'].unique()
gene_list = df_network['gene_name'].unique()

In [ ]:
adj_matrix = np.zeros((len(gene_list), len(gene_list)))
edge_weights = np.zeros((len(gene_list), len(gene_list)))

In [ ]:
# Create adjacency matrix and edge weight matrix

for i, row in enumerate(df_network.iterrows()):
    TF, gene, score = row[1]
    idx = np.argwhere(gene_list == TF)
    idy = np.argwhere(gene_list == gene)
    adj_matrix[idx, idy] = 1
    edge_weights[idx, idy] = score

In [ ]:
gene_exp_list = np.array(df_var['gene_short_name'])
gene_exp_array = np.zeros((len(gene_list), len(df_obs)))

In [ ]:
# Create node feature matrix

for i, gene in enumerate(gene_list):
    idx = np.argwhere(gene_exp_list == gene)
    gene_exp_array[i] = mat_np[idx]

In [ ]:
adj_tensor = torch.from_numpy(adj_matrix)
trait_tensor = torch.from_numpy(gene_exp_array)
edge_weight_tensor = torch.from_numpy(edge_weights)

In [ ]:
# Create torch_geometric graph dataset

edge_index = adj_tensor.nonzero().t()
data = Data(x=trait_tensor, edge_index=edge_index, edge_attr=edge_weight_tensor[edge_index[0], edge_index[1]])

## Run GCN training and test

In [ ]:
# Implementation of Edge regression

class EdgeRegression(torch.nn.Module):
    def __init__(self, num_node_features, hidden_features, out_features):
        super(EdgeRegression, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_features)
        self.conv2 = GCNConv(hidden_features, hidden_features)
        self.conv = GCNConv(hidden_features, out_features)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv(x, edge_index)
        
        return x
    
    def decode(self, x, edge_index):
        src, dst = edge_index
        score = (x[src] * x[dst]).sum(dim=-1)
        
        return score

In [ ]:
model = EdgeRegression(num_node_features=10000, hidden_features=200, out_features=10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

In [ ]:
def train(model, optimizer, data, criterion):
    model.train()
    optimizer.zero_grad()
    out = model.encode(data.x.float(), data.edge_index)
    score = model.decode(out, data.edge_index)
    loss = criterion(score, data.edge_attr.float())
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data, criterion):
    model.eval()
    with torch.no_grad():
        out = model.encode(data.x.float(), data.edge_index)
        score = model.decode(out, data.edge_index).view(-1)
        loss = criterion(score, data.edge_attr.float())
    return loss.item()

In [ ]:
torch.manual_seed(1234)
for epoch in range(50):
    loss = train(model, optimizer, data, criterion)
    print(f'Epoch {epoch + 1}, Loss: {loss:.4f}')
    if epoch % 10 == 9:
        loss = test(model, data, criterion)
        print(f'Test Loss: {loss:.4f}')

In [ ]:
model.eval()
with torch.no_grad():
    out = model.encode(data.x.float(), data.edge_index)
    score = model.decode(out, data.edge_index).view(-1)

## Visualization

### Visualization of graph

In [ ]:
g = torch_geometric.utils.to_networkx(data, to_undirected=True)

In [ ]:
fig = plt.figure()
nx.draw(g, nodelist=np.unique(np.array(g.edges).reshape(1, -1)), node_size=10, ax=fig.add_subplot())
fig.savefig("../output/EB1/test_graph.png")

### Visualization of UMAP projection of gene embedding

In [ ]:
reducer = umap.UMAP()
scaled_data = StandardScaler().fit_transform(out.numpy())

In [ ]:
embedding = reducer.fit_transform(scaled_data)

In [ ]:
TF_gene = [1 if x in np.unique(np.array(g.edges).reshape(2, -1)[0]) else 2 for x in range(embedding.shape[0])]

In [ ]:
fig = plt.figure()
plt.scatter(embedding[:, 0], embedding[:, 1], c=TF_gene)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of embedding of genes', fontsize=10)
plt.legend(labels=['TF', 'gene'])
fig.savefig("../output/EB1/test_embedding.png")

In [ ]:
model = EdgeRegression(num_node_features=10000, hidden_features=200, out_features=10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

### Visualize the training process

In [ ]:
training_loss = []
test_loss = []

torch.manual_seed(1234)
for epoch in range(50):
    loss = train(model, optimizer, data, criterion)
    training_loss.append(loss)
    if epoch % 10 == 9:
        loss = test(model, data, criterion)
        test_loss.append(loss)

In [ ]:
fig = plt.figure()
plt.plot(np.arange(1, 51), training_loss)
plt.scatter(np.arange(10, 60, 10), test_loss, c='orange')
plt.title('Training and test loss during training')
plt.legend(labels=['training', 'test'])
fig.savefig("../output/EB1/test_loss.png")